In [ ]:
StackedRipEvents = []
StackedRipTetrodes = []
for e, ep in enumerate(ep_list):
    # Stack all the rip events in preparation for counting overlaps
    StackedRipEvents.append(RippleEvents[e][0].copy())
    for k in range(1,len(TetrodeList)):
        StackedRipEvents[e] = StackedRipEvents[e] if (RippleEvents[e][k].size == 0) else \
                    np.vstack((StackedRipEvents[e], RippleEvents[e][k]))

    # Create a tetrode ID list for each event
    StackedRipTetrodes.append([[tetidx] for tetidx, E in enumerate(RippleEvents[e]) for _ in range(len(E)) ])

    # Now, sort events by start time
    if (StackedRipEvents[e].size > 0):
        sortidx = np.argsort(StackedRipEvents[e][:,0], axis=0)
        StackedRipTetrodes[e] = [StackedRipTetrodes[e][i] for i in sortidx]
        StackedRipEvents[e] = StackedRipEvents[e][sortidx,:]
    else:
        StackedRipEvents[e] = np.empty(shape=[0, 2])

def event_union(r, s):
    if ((r[1] < s[0]) | (s[1] < r[0])):
        return np.array([])
    else:
        return np.array([np.min([r[0], s[0]]), np.max([r[1], s[1]])])

RipEventList = []
RipEventTetrodes = []
for e in range(len(ep_list)):
    if (StackedRipEvents[e].size > 0):
        RipEventList.append(np.atleast_2d(StackedRipEvents[e][0,:]))
        RipEventTetrodes.append([StackedRipTetrodes[e][0]])
        for k, ev in enumerate(StackedRipEvents[e][1:,:]):
            overlap = event_union(ev, RipEventList[e][-1])
            if (overlap.size == 0):
                RipEventList[e] = np.vstack((RipEventList[e],ev))
                RipEventTetrodes[e].append([StackedRipTetrodes[e][k]])
            else:
                RipEventList[e][-1,:] = overlap
                RipEventTetrodes[e][-1] += StackedRipTetrodes[e][k]
    else:
        RipEventList.append(np.empty(shape=[0, 2]))
        RipEventTetrodes.append([])

        
RipEventList = [np.asarray(E) for E in RipEventList]
NumRipTetrodes = [np.asarray([len(NT) for NT in E]) for E in RipEventTetrodes]

Win = 0.015 # Require elevated power at least 15 ms (total over 3 tetrodes)

# Select for at least 3 tetrodes
RipEventList = [np.extract(E[:,1] - E[:,0] > np.round(FS[0]*Win) && NT > 2, E) for (E,NT) in 
                 zip(RipEventList, NumRipTetrodes)]
